<a href="https://colab.research.google.com/github/frank-895/machine_learning_journey/blob/main/gradient_boosting_machine/gbm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [34]:
import pandas as pd
from fastai.tabular.all import *
from fastai.data.transforms import IndexSplitter

# Gradient Boosting Machines - Forecasting Sticker Sales

## Introduction

After learning about **ensembles of decision trees** as an approach to modelling tabular data, I've decided to have another shot at the [Forecasting Sticker Sales](https://www.kaggle.com/competitions/playground-series-s5e1) Kaggle competition.

While I initially approached the problem with **deep learning** and scored in the top 27% of predictions, I want to see if I can improve on this by taking a different approach.

In my last notebook I uncovered **random forests** which is one approach of ensembling decision trees. However, there is another area of ensembling I wish to learn more about, called **gradient boosting machines (GBMs)**.

Since I've already gone into detail in my [previous notebook](https://github.com/frank-895/machine_learning_journey/tree/main/random_forests) about how decision trees are made and what ensembling is, I will be focusing on *building* that knowledge. I won't go into detail about when to use **ensembles of decision trees** vs **deep learning**.

Furthermore, I've selected a **regression** problem for this project, as I practiced a **classification** problem when learning random forests.

The first thing to establish is the key differences between **random forests** and **GBMs**.

Where random forests utilise **bagging** (training each model on a different data subset and averaging), GBMs use **boosting**, where models are 'added'. Each new decision tree correct the errors made by the previous models and the final prediction is the the weighted sum of all the predictions from all the models.

GBMs can generally perform better than random forests, but they are more complicated to implement.
- They require some fussing with hyperparamaters.
- They are prone to overfitting as each tree is not independent of the others.
- They are rely on sequential training, which is slower, whereas random forests can utilise parallel training (as they are independent trees).

So, a good approach would be to start with a random forest, then try a GBM. We will do that here, but the description for random forests will be brief as we have already gone into detail about this previously.

## Data Engineering

Let's start with our data.

In [35]:
df = pd.read_csv('train.csv')
df.isnull().sum()

,0
id,0
date,0
country,0
store,0
product,0
num_sold,8871


We will drop training rows missing the independent variable to focus on learning the concept of GBMs.

Normally, with features missing values (there are none here) we would fill numerical features with the mean/median/mode and categorical features with a placeholder like 'Unknown' as the fact that data is missing can still be useful information to train a model.

In [36]:
df.dropna(axis=0, inplace=True)
df

,id,date,country,store,product,num_sold
1,1,2010-01-01,Canada,Discount Stickers,Kaggle,973.0
2,2,2010-01-01,Canada,Discount Stickers,Kaggle Tiers,906.0
3,3,2010-01-01,Canada,Discount Stickers,Kerneler,423.0
4,4,2010-01-01,Canada,Discount Stickers,Kerneler Dark Mode,491.0
5,5,2010-01-01,Canada,Stickers for Less,Holographic Goose,300.0
...,...,...,...,...,...,...
230125,230125,2016-12-31,Singapore,Premium Sticker Mart,Holographic Goose,466.0
230126,230126,2016-12-31,Singapore,Premium Sticker Mart,Kaggle,2907.0
230127,230127,2016-12-31,Singapore,Premium Sticker Mart,Kaggle Tiers,2299.0
230128,230128,2016-12-31,Singapore,Premium Sticker Mart,Kerneler,1242.0


Now, we will perform much of the same **feature engineering** as the deep learning approach. Since tree-based models are robust to non-normality, log transformations will not be used in this approach on any predictors; however, we will perform a log transformation on the `num_sold` column as our predictions are highly skewed with a long right tail (shown in the last notebook).

As previously discussed, OHE is generally not needed for tree-based approaches.

In [37]:
%%capture
df['num_sold'] = np.log1p(df['num_sold'])

# extract useful features from date
add_datepart(df, 'date')

# drop unneeded columns
df.drop('id', axis=1, inplace=True)

Now, we can define a training and validation set. Since we are working with data over time, we will clip the end of the df and use this as our **validation set**.

I am going to be using a new techinque I recently learn for this, defined by `TabularPandas`. `TabularPandas` sits on top of a pandas df and is provided by `fastai`. This class modifies objects in place and runs transformations when data is passed in, rather than lazily as data is accessed.

In [38]:
procs = [Categorify, FillMissing]

These two transformations are not strictly necessary on this dataset as we have no missing values in the predictor columns. However, `FillMissing` is a `TabularProc` (much like a `Transform` in pandas) that  replaces missing values with median values. `Categorify` is a new feature that is set `True` for any row where the value was missing (as missing values can hold valuable information for training). We will pass the `TabularProc`s when creating our training and validation sets.

In [39]:
cond = df.Year<2016
train_idx, valid_idx = np.where(cond)[0], np.where(~cond)[0]
splits = (list(train_idx),list(valid_idx))

`where()` is a useful function returning indices of all True values.

We also want to define our **categorical** and **continuous** variables before creating our `TabularPandas`. While we did this manually last time, we can actually benefit from the `fastai` function `cont_cat_split` that can automtically infer the variable type.

In [40]:
cont, cat = cont_cat_split(df, dep_var="num_sold")
cont, cat

(['Week', 'Day', 'Dayofyear', 'Elapsed'],
 ['country',
  'store',
  'product',
  'Year',
  'Month',
  'Dayofweek',
  'Is_month_end',
  'Is_month_start',
  'Is_quarter_end',
  'Is_quarter_start',
  'Is_year_end',
  'Is_year_start'])

Now we are ready for our `TabularPandas`!

In [41]:
data = TabularPandas(df, procs, cat, cont, y_names="num_sold", splits=splits)
len(data.train), len(data.valid)

(189492, 31767)

## Random Forest

Let's build a random forest! While I have previously discussed in-depth how **random forests** work, I am now using a random forest for a **regression** task for the first time.

**How does the decision tree predict a continuous value?**

Each decision tree will output a numerical value for a data point, which is the mean of the target values of the training samples in the leaf node where the data point falls.

**But how does the model decide where to make the split at each binary split?**

The model will calculate and minimise the **variance** also known as **minimizing impurity**. Just like a classification task, the model will see how similar the targets of each data point are for each possible split. Instead of just comparing if they are the same or different though, it will use a formula called the **weighted average variance**. This basically means that values that are closer together will have a lower **impurity** and this split will be prioritised.

This explains how a decision tree performs a regression task as opposed to a classification task.

In [42]:
from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor(n_jobs=-1, # no of jobs to run in parallel, -1 means use all CPUs
                           n_estimators=100, # no trees
                           max_features=0.5, # no of columns to sample at each split point
                           min_samples_leaf=3, # min number of data points in leaf nodes
                           max_samples=0.75, # how many rows to sample for each training tree
                           oob_score=True) # we use oob to claculate the out-of-bag error later

rf.fit(data.train.xs, data.train.y)
y_pred = rf.predict(data.valid.xs)

Now that we have our predictions, let's evaluate them using **mean absolute percentage error** (MAPE), which is the defined metric for this Kaggle competition.

In [43]:
import numpy as np

def mape(y_actual, y_pred, epsilon=1e-10):
  y_actual = np.array(y_actual)
  y_pred = np.array(y_pred)

  mape = np.mean(np.abs((y_actual - y_pred) / (y_actual + epsilon)))
  return mape

mape(data.valid.y, y_pred)

0.020203011469208002

At this point, I'm going to make some predictions on the test dataset and submit to Kaggle, just to see where we are at!

I ran into some issues here, as the data pipeline for the model requires a `TabularPandas`. Since it appears this class was designed for neural networks with fastai - which inherintly hold information about the data formatting - I can't seem to find a way to add a test set to my pre-exisiting `TabularPandas` object. The code below is a bit messy, and the test_data is stored in `.train`. If anyone has a better way of approaching this, I would love to know!

In [45]:
%%capture
test_df = pd.read_csv('test.csv')
add_datepart(test_df, 'date')

test_data = TabularPandas(test_df, procs, cat, cont)

test_df['num_sold'] = rf.predict(test_data.train.xs)
test_df['num_sold'] = np.expm1(test_df['num_sold'])
sub_df = test_df[['id', 'num_sold']]

sub_df.to_csv('submission.csv', index=False)

So, after submitting the results of our predictions on the test data, we were scoring a MAPE of 0.162! This is not a bad prediction considering our deep learning result was sitting around this prior to optimisation!

## Gradient Boosting Machine

I can now try out the `GradientBoostingRegressor` also provided by sklearn.

In [51]:
from sklearn.ensemble import GradientBoostingRegressor

gbm = GradientBoostingRegressor(
    n_estimators = 100,
    learning_rate=1,
    max_depth=20,
    min_samples_split=2,
    min_samples_leaf=3
)

gbm.fit(data.train.xs, data.train.y)

y_pred = gbm.predict(data.valid.xs)

mape(data.valid.y, y_pred)

0.02299528907296866

As you can see above, the model requires a lot more playing around with hyperparameters and it also took significantly longer to train.

The **learning rate** in a GBM is used to scale the contribution of each new weak decision tree to the final model. A smaller learning rate means smaller updates to the model, while a larger learning rate means larger updates to the model. You use the learning rate in GBMs much like you would in a NN - you don't want the value to big or too small.

In [52]:
%%capture
test_df = pd.read_csv('test.csv')
add_datepart(test_df, 'date')

test_data = TabularPandas(test_df, procs, cat, cont)

test_df['num_sold'] = rf.predict(test_data.train.xs)
test_df['num_sold'] = np.expm1(test_df['num_sold'])
sub_df = test_df[['id', 'num_sold']]

sub_df.to_csv('submission2.csv', index=False)

I used the model on the test set (as above) and after playing around with the hyperparameters I was able to get a MAPE value of 0.164, slightly worse than the random forest.

While we weren't able to improve on the result generated by our NN in the competition, there is other useful information we can derive from our **ensembles of decision trees**.

## Conclusion